In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import time
import numpy as np
import scipy.sparse as sparse
import random as random
import os
import sys
import copy

# PyTorch Geometric imports
try:
    from torch_geometric.nn import GCNConv
    from torch_geometric.utils import from_scipy_sparse_matrix
    print("PyTorch Geometric利用可能")
except ImportError:
    print("PyTorch Geometricがインストールされていません")
    print("pip install torch-geometric でインストールしてください")
    sys.exit(1)

# 再現性のための乱数シード固定
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    if "torch" in sys.modules:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(42)

print("ライブラリのインポート完了")
print(f"PyTorchバージョン: {torch.__version__}")
print(f"デバイス: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


In [ ]:
def load_data():
    print("Loading Cora dataset...")
    raw_nodes_data = np.genfromtxt('/content/drive/My Drive/Colab Notebooks/cora.content', dtype="str")
    print(raw_nodes_data)
    raw_node_ids = raw_nodes_data[:, 0].astype('int32')  # 各行の一列目に格納されてるノードIDを抽出
    raw_node_labels = raw_nodes_data[:, -1]# 各行の最終列に格納されてるラベルを抽出．このラベルが予測ターゲット

    unique = list(set(raw_node_labels))
    labels_enumerated = np.array([unique.index(label) for label in raw_node_labels])
    node_features = sparse.csr_matrix(raw_nodes_data[:, 1:-1], dtype="float32")

    ids_ordered = {raw_id: order for order, raw_id in enumerate(raw_node_ids)} #実際のノードIDを0から節点数-1に対応付け
    raw_edges_data = np.genfromtxt('/content/drive/My Drive/Colab Notebooks/cora.cites', dtype="int32")
    edges = np.array(list(map(ids_ordered.get, raw_edges_data.flatten())), dtype='int32').reshape(raw_edges_data.shape) # 実際のノードIDを変換. reshapeでデータ構造を元の枝ファイルと同様に変更．

    adj = sparse.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                            shape=(labels_enumerated.shape[0], labels_enumerated.shape[0]),
                            dtype=np.float32)

    adj = adj + adj.T.multiply(adj.T > adj) #隣接行列を対象に変更 (つまり，無向グラフに変換)
    adj = adj + sparse.eye(adj.shape[0]) #対角成分に1を挿入

    node_degrees = np.array(adj.sum(1)) #列毎の総和を計算する（つまり，次数を計算する）
    node_degrees = np.power(node_degrees, -0.5).flatten()
    degree_matrix = sparse.diags(node_degrees, dtype=np.float32)
    print(degree_matrix)

    adj = degree_matrix @ adj @ degree_matrix #行列の積を計算．
    print(adj)

    features = torch.FloatTensor(node_features.toarray())
    labels = torch.LongTensor(labels_enumerated)
    adj = torch.FloatTensor(np.array(adj.todense()))

    return features, labels, adj, edges


In [ ]:
def visualize_embedding_tSNE(labels, y_pred, num_classes):
    cora_label_to_color_map = {0: "red", 1: "blue", 2: "green", 3: "orange", 4: "yellow", 5: "pink", 6: "gray"}

    node_labels = labels.cpu().numpy()
    out_features = y_pred.detach().cpu().numpy()
    t_sne_embeddings = TSNE(n_components=2, perplexity=30, method='barnes_hut').fit_transform(out_features)

    plt.figure(figsize=(10, 8))
    for class_id in range(num_classes):
        plt.scatter(t_sne_embeddings[node_labels == class_id, 0],
                    t_sne_embeddings[node_labels == class_id, 1], s=20,
                    color=cora_label_to_color_map[class_id],
                    edgecolors='black', linewidths=0.15,
                    label=f'Class {class_id}')

    plt.axis("off")
    plt.title("PyTorch Geometric GCN - t-SNE可視化", fontsize=14)
    plt.legend()
    plt.show()


In [ ]:
class Net(nn.Module):
    """
    senadkurtisi/pytorch-GCN参考のPyTorch Geometric GCN実装
    
    アーキテクチャ:
    - 第1層: GCNConv(num_features, 16, cached=True) + ReLU
    - Dropout(p=0.5)
    - 第2層: GCNConv(16, num_classes, cached=True)
    - log_softmax
    """
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_features, 16, cached=True)
        self.conv2 = GCNConv(16, num_classes, cached=True)
        
        # パラメータ数計算用
        self.num_features = num_features
        self.num_classes = num_classes
        
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
    def count_parameters(self):
        """パラメータ数を計算"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# EarlyStoppingクラス
class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-4, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        
        self.best_loss = None
        self.counter = 0
        self.best_weights = None
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            if self.restore_best_weights:
                self.best_weights = copy.deepcopy(model.state_dict())
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.restore_best_weights:
                self.best_weights = copy.deepcopy(model.state_dict())
        else:
            self.counter += 1
            
        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False

def accuracy(output, labels):
    y_pred = output.max(1)[1].type_as(labels)
    correct = y_pred.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

print("PyTorch Geometric GCNモデル定義完了")


In [ ]:
# データロード（既存Notebookと完全一致）
features, labels, adj, edges = load_data()

print(f"データセット情報:")
print(f"  ノード数: {features.shape[0]}")
print(f"  特徴量次元: {features.shape[1]}")
print(f"  クラス数: {labels.max().item() + 1}")
print(f"  エッジ数: {edges.shape[0]}")

# PyTorch Geometric用にエッジインデックス形式に変換
# 隣接行列からエッジインデックスを抽出
adj_sparse = sparse.coo_matrix(adj.numpy())
edge_index, _ = from_scipy_sparse_matrix(adj_sparse)

print(f"  エッジインデックス形状: {edge_index.shape}")
print(f"  エッジインデックス型: {edge_index.dtype}")

# データ分割（既存Notebookと完全一致）
num_classes = int(labels.max().item() + 1)
train_size_per_class = 20
validation_size = 500
test_size = 1000
classes = [ind for ind in range(num_classes)]
train_set = []

# 各クラス20サンプルで訓練セット構築
for class_label in classes:
    target_indices = torch.nonzero(labels == class_label, as_tuple=False).tolist()
    train_set += [ind[0] for ind in target_indices[:train_size_per_class]]

# 残りのサンプルを検証・テストセットに分割
validation_test_set = [ind for ind in range(len(labels)) if ind not in train_set]
validation_set = validation_test_set[:validation_size]
test_set = validation_test_set[validation_size:validation_size+test_size]

print(f"\nデータ分割:")
print(f"  訓練セット: {len(train_set)} サンプル")
print(f"  検証セット: {len(validation_set)} サンプル")
print(f"  テストセット: {len(test_set)} サンプル")


In [ ]:
# PyTorch Geometric GCNモデルの作成
model = Net(num_features=features.shape[1], num_classes=num_classes)

print("=== PyTorch Geometric GCNモデル ===")
print(model)
print(f"\n総パラメータ数: {model.count_parameters():,}")

# 各層のパラメータ数詳細
print(f"\n=== 層別パラメータ数 ===")
print(f"入力次元: {features.shape[1]}")
print(f"隠れ層次元: 16")
print(f"出力次元: {num_classes}")
print(f"第1層 GCNConv: {features.shape[1]} -> 16")
print(f"第2層 GCNConv: 16 -> {num_classes}")

# senadkurtisi/pytorch-GCN参考の学習設定
lr = 0.01
weight_decay = 5e-4
max_epochs = 200
patience = 10

print(f"\n=== 学習設定（senadkurtisi/pytorch-GCN準拠） ===")
print(f"学習率: {lr}")
print(f"重み減衰: {weight_decay}")
print(f"最大エポック数: {max_epochs}")
print(f"Early Stopping patience: {patience}")

# デバイス設定
if torch.cuda.is_available():
    print("CUDA利用可能 - GPUで学習を実行")
    model.cuda()
    edge_index = edge_index.cuda()
    features = features.cuda()
    labels = labels.cuda()
else:
    print("CPUで学習を実行")

# オプティマイザーと損失関数
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

# Early Stopping初期化
early_stopping = EarlyStopping(patience=patience, min_delta=1e-4, restore_best_weights=True)

print(f"最適化器: Adam (lr={lr}, weight_decay={weight_decay})")
print(f"損失関数: CrossEntropyLoss")
print("=" * 50)


In [ ]:
# 学習履歴記録用
validation_acc = []
validation_loss = []
training_acc = []
training_loss = []

print("=== PyTorch Geometric GCN学習開始 ===")
print(f"最大エポック数: {max_epochs}")
print(f"Early Stopping: 有効 (patience={patience})")
print("=" * 50)

# 学習時間計測開始
t_start = time.time()
stopped_early = False

for epoch in range(max_epochs):
    # 訓練フェーズ
    model.train()
    optimizer.zero_grad()
    
    output = model(features, edge_index)
    train_loss = criterion(output[train_set], labels[train_set])
    train_acc = accuracy(output[train_set], labels[train_set])
    
    train_loss.backward()
    optimizer.step()
    
    # 検証フェーズ
    model.eval()
    with torch.no_grad():
        output = model(features, edge_index)
        val_loss = criterion(output[validation_set], labels[validation_set])
        val_acc = accuracy(output[validation_set], labels[validation_set])
    
    # 履歴記録
    training_loss.append(train_loss.item())
    training_acc.append(train_acc.item())
    validation_loss.append(val_loss.item())
    validation_acc.append(val_acc.item())
    
    # Early Stopping チェック
    if early_stopping(val_loss, model):
        print(f"EARLY STOPPING: エポック {epoch+1} で停止")
        stopped_early = True
        break
    
    # 進捗表示（20エポック毎）
    if epoch % 20 == 0:
        print(f"Epoch: {epoch:4d} | Train loss: {train_loss.item():.3f} | "
              f"Train acc: {train_acc:.3f} | Val loss: {val_loss.item():.3f} | "
              f"Val acc: {val_acc:.3f}")

# 学習時間計測終了
t_end = time.time()

if not stopped_early:
    print(f"学習完了: 全 {max_epochs} エポック実行")
    
print(f"学習時間: {t_end-t_start:.2f}秒")
print(f"最終検証精度: {validation_acc[-1]:.3f}")
print("=" * 50)


In [ ]:
# 最終テスト精度の評価
print("=== 最終テスト結果 ===")

with torch.no_grad():
    model.eval()
    output = model(features, edge_index)
    test_loss = criterion(output[test_set], labels[test_set])
    test_acc = accuracy(output[test_set], labels[test_set])

print(f"Test loss: {test_loss:.3f}")
print(f"Test accuracy: {test_acc:.3f}")
print(f"使用したモデル: PyTorch Geometric GCN (2層)")
print(f"パラメータ数: {model.count_parameters():,}")
print(f"学習時間: {t_end-t_start:.2f}秒")
print(f"学習エポック数: {len(training_acc)}")

# 結果サマリー
print(f"\n=== 結果サマリー ===")
print(f"最終訓練精度: {training_acc[-1]:.3f}")
print(f"最終検証精度: {validation_acc[-1]:.3f}")
print(f"最終テスト精度: {test_acc:.3f}")
print(f"参考実装: senadkurtisi/pytorch-GCN (目標: 82%)")

# senadkurtisi実装との比較
target_accuracy = 0.82
if test_acc >= target_accuracy:
    print(f"✓ 目標精度 {target_accuracy:.1%} を達成！")
else:
    print(f"△ 目標精度 {target_accuracy:.1%} まで {target_accuracy - test_acc:.3f} 不足")

print("=" * 50)


In [ ]:
# 学習曲線の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 損失曲線
ax1.plot(training_loss, label='Training Loss', color='blue', alpha=0.7)
ax1.plot(validation_loss, label='Validation Loss', color='red', alpha=0.7)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('PyTorch Geometric GCN - 学習曲線 (Loss)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 精度曲線
ax2.plot(training_acc, label='Training Accuracy', color='blue', alpha=0.7)
ax2.plot(validation_acc, label='Validation Accuracy', color='red', alpha=0.7)
ax2.axhline(y=test_acc.cpu().numpy(), color='green', linestyle='--', 
           label=f'Test Accuracy ({test_acc:.3f})', alpha=0.8)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('PyTorch Geometric GCN - 学習曲線 (Accuracy)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 学習統計
print(f"=== 学習統計 ===")
print(f"最高検証精度: {max(validation_acc):.3f} (エポック {np.argmax(validation_acc)+1})")
print(f"最低検証損失: {min(validation_loss):.3f} (エポック {np.argmin(validation_loss)+1})")
print(f"最終エポック: {len(training_acc)}")

if stopped_early:
    print(f"Early Stopping発動: {patience}エポック改善なしで停止")
else:
    print(f"全エポック完了: {max_epochs}エポック実行")

print("=" * 50)


In [ ]:
# t-SNE可視化の実行
print("=== t-SNE可視化実行中 ===")
print("学習済みモデルの出力を2次元に埋め込み中...")

with torch.no_grad():
    model.eval()
    output = model(features, edge_index)
    
# t-SNE可視化
visualize_embedding_tSNE(labels, output, num_classes)

print("t-SNE可視化完了")
print("各色は異なるクラスを表しています")
print("クラスタリングの品質が高いほど、同色の点が密集します")
print("=" * 50)


In [ ]:
print("=== PyTorch Geometric GCN実装詳細 ===")
print()

print("【参考実装】")
print("- senadkurtisi/pytorch-GCN: https://github.com/senadkurtisi/pytorch-GCN")
print("- PyTorch Geometric: https://github.com/pyg-team/pytorch_geometric")
print()

print("【モデルアーキテクチャ】")
print("class Net(nn.Module):")
print("    def __init__(self, num_features, num_classes):")
print("        self.conv1 = GCNConv(num_features, 16, cached=True)")
print("        self.conv2 = GCNConv(16, num_classes, cached=True)")
print()
print("    def forward(self, x, edge_index):")
print("        x = F.relu(self.conv1(x, edge_index))")
print("        x = F.dropout(x, p=0.5, training=self.training)")
print("        x = self.conv2(x, edge_index)")
print("        return F.log_softmax(x, dim=1)")
print()

print("【技術的特徴】")
print("1. PyTorch Geometric GCNConv:")
print("   - 効率的なスパース行列演算")
print("   - cached=True による計算最適化")
print("   - メッセージパッシング最適化")
print()

print("2. シンプルな2層構成:")
print(f"   - 入力: {features.shape[1]}次元 → 隠れ: 16次元 → 出力: {num_classes}次元")
print("   - ReLU活性化関数")
print("   - Dropout(p=0.5)による正則化")
print()

print("3. 学習設定:")
print(f"   - Adam最適化器 (lr={lr}, weight_decay={weight_decay})")
print(f"   - Early Stopping (patience={patience})")
print(f"   - 最大エポック数: {max_epochs}")
print()

print("【PyTorch Geometricの利点】")
print("- 最適化されたGNN実装")
print("- GPU加速対応")
print("- メモリ効率的なスパース演算")
print("- 豊富なGNN層の提供")
print("- 活発な開発コミュニティ")
print()

print("【senadkurtisi実装との比較】")
print(f"- 目標精度: 82% (Mean: 81.0%, Std: 1.0%)")
print(f"- 本実装精度: {test_acc:.1%}")
print(f"- 学習時間: {t_end-t_start:.2f}秒")
print(f"- パラメータ数: {model.count_parameters():,}")
print()

print("【データ処理の特徴】")
print("- エッジインデックス形式への変換")
print("- PyTorch Geometric形式のデータ構造")
print("- 効率的なグラフ表現")
print("- cached=True による前処理最適化")

print("=" * 60)
